In [1]:
from src.data_reader import DataReader
from src.constant import *
import matplotlib.pyplot as plt
import numpy as np
from src.datasets.dataset import Dataset
from src.models.itemknn.itemknn import ItemKNN
import pandas as pd
from src.evaluation import compute_mrr

In [2]:
dataset = Dataset()

In [3]:
split_dict = dataset.get_split()

In [4]:
train, train_label = split_dict[TRAIN]

In [5]:
val, val_label = split_dict[VAL]

In [6]:
lead_data = dataset.get_test_leaderboard_sessions()

In [19]:
model = ItemKNN(dataset, topk=2000, normalization=False, t1=1.0, t2=1.0, c=0.5, l=0.5, time_weight=100)

In [23]:
model.compute_similarity_matrix(train)

Using Time Weight on Interaction matrix


Done: 100%|██████████| 23691/23691 [00:01<00:00, 22010.12it/s]


In [21]:
recs = model.recommend(
    interactions=val,
    remove_seen=True,
    cutoff=100,
    leaderboard=False
)

Using Time Weight on Interaction matrix


In [22]:
compute_mrr(recs, val_label)

MRR: 0.12002561320268756


0.12002561320268756

In [ ]:
train

In [ ]:
train["last_buy"] = train.groupby(SESS_ID)[
    DATE
].transform(max)
train["first_buy"] = train.groupby(SESS_ID)[
    DATE
].transform(min)

In [ ]:
train

In [ ]:
train["time_score"] = 1/((
    (train["last_buy"]-train[DATE]).apply(lambda x: x.total_seconds()/3600)
)+1)

In [ ]:
train

In [ ]:
train["time_score"].min()

In [ ]:
(train[DATE] - train["first_buy"]).apply(lambda x: x.total_seconds()/60)